# 1 часть модели, способная предсказывать необходимый параметр по SMILES.
**Примечание:** Количество SMILES для одного масла может быть произвольным;

In [ ]:
%pip install tensorflow==2.15 deepchem

In [19]:
import deepchem as dc
import polars as pl
import pandas as pd
import numpy as np

from rdkit import Chem, RDLogger

from typing import Optional

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


def read(n: str, **kwargs):
    return pl.read_csv(f"../data/{n}", **kwargs).to_pandas()


RDLogger.DisableLog("rdApp.*")

public = read("public/data.csv")
check = read("public/check.csv")
check = check.drop_duplicates(subset=["blend_id"]).reset_index(drop=True)

In [20]:
featurizer = dc.feat.ConvMolFeaturizer()


def preprocess(
    data: pd.DataFrame, preprocessor: Optional[ColumnTransformer] = None
) -> tuple[pd.DataFrame, Optional[pd.DataFrame], Optional[ColumnTransformer]]:
    data = data.copy()
    data.dropna(subset=["smiles"], inplace=True)

    def compute_descriptors(smiles: str):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None

        feature = featurizer.featurize([mol])[0]

        return {"ConvMol": feature}

    descriptors_df = pd.DataFrame.from_records(
        data.smiles.apply(compute_descriptors).dropna()
    )
    data = pd.concat([data, descriptors_df], axis=1)
    data = data.drop("smiles", axis=1)

    data.dropna(subset=["blend_id"], inplace=True)
    categorical_cols = ["blend_id"]
    if (
        preprocessor is None
        and len(data.select_dtypes(include=["object", "category"]).columns) > 0
    ):
        categorical_transformer = Pipeline(
            steps=[
                (
                    "imputer",
                    SimpleImputer(strategy="constant", fill_value="missing"),
                ),
                ("onehot", OneHotEncoder(handle_unknown="ignore")),
            ]
        )
        preprocessor = ColumnTransformer(
            transformers=[
                ("cat", categorical_transformer, categorical_cols),
            ],
            remainder="passthrough",
        )

        encoding_data = pd.DataFrame(
            preprocessor.fit_transform(data[categorical_cols]).toarray(),
            columns=preprocessor.get_feature_names_out(categorical_cols),
        )
        data = pd.concat([data, encoding_data], axis=1)
        data = data.drop(categorical_cols, axis=1)
    else:
        encoding_data = pd.DataFrame(
            preprocessor.transform(data[categorical_cols]).toarray(),
            columns=preprocessor.get_feature_names_out(categorical_cols),
        )
        data = pd.concat([data, encoding_data], axis=1)
        data = data.drop(categorical_cols, axis=1)

    data.dropna(inplace=True)

    if "oil_property_param_value" in data.columns:
        y = data.pop("oil_property_param_value")
        return pd.DataFrame(data), pd.DataFrame(y), preprocessor

    return pd.DataFrame(data), None, preprocessor


X, y, preprocessor = preprocess(public)
X.head(1)

,ConvMol,cat__blend_id_007b4d7a-b1f8-11ed-93b4-005056921581,cat__blend_id_0109d480-f91d-11ec-96ac-005056921581,cat__blend_id_01bc9ed0-0f2f-11ed-9900-005056921581,cat__blend_id_01d12ce2-ddec-11ed-8992-005056921581,cat__blend_id_01eae8bc-1885-11ed-98cd-005056921581,cat__blend_id_0366a302-d9cd-11ed-96ad-005056921581,cat__blend_id_037e9272-e5a9-11ed-8950-005056921581,cat__blend_id_03f8d734-5441-11ed-993d-005056921581,cat__blend_id_043e4af2-a2fc-11ee-850f-005056921581,...,cat__blend_id_f9a8223a-0cf1-11ed-87b0-005056921581,cat__blend_id_fb57b4fe-632f-11ec-8eb2-005056921581,cat__blend_id_fb583e86-4de6-11ed-9a6d-005056921581,cat__blend_id_fbc44952-0f2d-11ed-9900-005056921581,cat__blend_id_fc68e5c8-b25f-11ec-8ff7-005056921581,cat__blend_id_fd893206-5e3a-11ec-803b-005056921581,cat__blend_id_fd96a6a2-b0fb-11ec-9d4e-005056921581,cat__blend_id_fe2c1c0a-4f84-11ed-93f9-005056921581,cat__blend_id_fe497da4-4ac5-11ed-8f31-005056921581,cat__blend_id_fe8635a0-1262-11ed-8f47-005056921581
0,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
dataset = dc.data.NumpyDataset(X=X.ConvMol, y=y.oil_property_param_value)

splitter = dc.splits.RandomSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(
    dataset=dataset, frac_train=0.9, frac_valid=0.0, frac_test=0.1
)

In [42]:
model = dc.models.GraphConvModel(n_tasks=1, mode="regression")

# Model training
model.fit(train_dataset)

10523371110.4

In [43]:
# initialze the metric
metric = dc.metrics.Metric(dc.metrics.mae_score)

# evaluate the model
valid_score = model.evaluate(valid_dataset or test_dataset, [metric])["mae_score"]
test_score = model.evaluate(test_dataset, [metric])["mae_score"]

print(f"Valid score: {valid_score:.2f}, Test score: {test_score:.2f}")

Valid score: 75848.51, Test score: 75848.51


In [47]:
X_check, _, _ = preprocess(check, preprocessor)
X_check_dataset = dc.data.NumpyDataset(X=X_check.ConvMol)

predictions = pd.DataFrame(model.predict(X_check_dataset), columns=["results"])
predictions = pd.concat([check.blend_id, predictions], axis=1)
predictions.head(1)

,blend_id,results
0,d272c9a6-3332-11ed-9685-005056921581,29.260405


In [48]:
predictions.to_csv("../predictions.csv", index=False)

In [50]:
!7z a ../solution.zip graph_conv-smiles2target.ipynb ../predictions.csv


7-Zip 23.01 (x64) : Copyright (c) 1999-2023 Igor Pavlov : 2023-06-20

Scanning the drive:
2 files, 30168 bytes (30 KiB)

Creating archive: ../solution.zip

Add new data to archive: 2 files, 30168 bytes (30 KiB)


Files read from disk: 2
Archive size: 11898 bytes (12 KiB)
Everything is Ok
